In [21]:
import tensorflow as tf

In [3]:
import urllib.request

url="https://storage.googleapis.com/learning-datasets/rps.zip"
urllib.request.urlretrieve(url,"rps.zip")

url2="https://storage.googleapis.com/learning-datasets/rps-test-set.zip"
urllib.request.urlretrieve(url2,'rps-test-set.zip')

('rps-test-set.zip', <http.client.HTTPMessage at 0x25f06dd6fd0>)

In [52]:
import zipfile

with zipfile.ZipFile('rps.zip','r') as zip_ref:
    zip_ref.extractall('rps')

with zipfile.ZipFile('rps-test-set.zip','r') as zip_ref:
    zip_ref.extractall('rps-test-set')

In [53]:
training_dir="rps"

In [79]:
validation_dir='rps-test-set'

In [80]:
validation_datagen=ImageDataGenerator(rescale=1/255)

In [81]:
validation_generator=validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    class_mode='categorical'
)

Found 372 images belonging to 1 classes.


In [82]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
training_datagen=ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [83]:
training_generator=training_datagen.flow_from_directory(
    training_dir,
    target_size=(150,150),
    class_mode='categorical'
)

Found 2520 images belonging to 1 classes.


In [84]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')
])

In [89]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [90]:
history=model.fit(
    training_generator,
    epochs=25,
    validation_data=validation_generator,
    verbose=1
)

Epoch 1/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 78s 960ms/step - accuracy: 0.9873 - loss: 0.0144 - val_accuracy: 1.0000 - val_loss: 3.1757e-07
Epoch 2/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 76s 958ms/step - accuracy: 1.0000 - loss: 4.9699e-07 - val_accuracy: 1.0000 - val_loss: 4.1018e-08
Epoch 3/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 75s 941ms/step - accuracy: 1.0000 - loss: 2.1268e-07 - val_accuracy: 1.0000 - val_loss: 1.5382e-08
Epoch 4/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 83s 961ms/step - accuracy: 1.0000 - loss: 1.3690e-07 - val_accuracy: 1.0000 - val_loss: 3.5250e-09
Epoch 5/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 76s 959ms/step - accuracy: 1.0000 - loss: 8.5339e-08 - val_accuracy: 1.0000 - val_loss: 9.6137e-10
Epoch 6/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 76s 961ms/step - accuracy: 1.0000 - loss: 6.9018e-08 - val_accuracy: 1.0000 - val_loss: 3.2046e-10
Epoch 7/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 76s 958ms/step - accuracy: 1.0000 - loss: 6.9160e-08 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 76s 957

In [95]:
import numpy as np
from keras.preprocessing import image

fn='Mtest.jpg'
path = fn
img = image.load_img(path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x=x/255.0
prediction=model.predict(x)
classes=['paper','rock','scissor']
result=classes[np.argmax(prediction)]

print("prediction: ",result)
print("confidence: ",np.max(prediction))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
prediction:  rock
confidence:  1.0


In [97]:
model.save('rps_model.keras')

In [99]:
from tensorflow.keras.models import load_model
model=load_model('rps_model.keras')